### Suncal-Web Basic Measurement Risk Calculator

---

In [1]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import ipywidgets as widgets

import suncal
from suncal import risk
suncal.plotting.plotstyle.update({'font.size': 9})

In [3]:
r = risk.Risk()
r.set_itp(.95)
r.set_tur(4)

In [4]:
style = {'description_width': 'initial'}

tableout = widgets.Output()

plt.ioff()
fig = plt.figure()
plt.ion()

witp = widgets.FloatSlider(95, min=1, max=99, step=1, description='In-Tolerance Probability %', style=style)
wtur = widgets.FloatSlider(4, min=.1, max=20, step=.1, description='Test Uncertainty Ratio', style=style)
wgbf = widgets.FloatSlider(1, min=0, max=2, step=.01, description='Guardband Factor', style=style)
wmeas = widgets.FloatSlider(0, min=-2, max=2, step=.01, description='Measured Value', style=style)

def update_output():
    with tableout:
        tableout.clear_output(wait=True)
        display(r.out)
    r.out.plot_joint(plot=fig)

def on_itpchange(change):
    r.set_itp(change['new']/100)
    update_output()

def on_turchange(change):
    r.set_tur(change['new'])
    update_output()

def on_gbfchange(change):
    r.set_gbf(change['new'])
    update_output()

def on_measchange(change):
    r.set_testmedian(change['new'])
    update_output()

witp.observe(on_itpchange, names='value')
wtur.observe(on_turchange, names='value')
wgbf.observe(on_gbfchange, names='value')
wmeas.observe(on_measchange, names='value')

def on_gbmethod(change):
    if gbmethod.value == 'Target PFA':
        gbpfa.layout.visibility = 'visible'
    else:
        gbpfa.layout.visibility = 'hidden'
        
def on_gbapply(change):
    if gbmethod.value == 'RSS':
        r.calc_guardband('rss')
    elif gbmethod.value == 'Method 6':
        r.calc_guardband('dobbert')
    else:
        r.calc_guardband('pfa', pfa=gbpfa.value/100)
    wgbf.value = r.get_gbf()
    update_output()


gbmethod = widgets.Dropdown(options=['RSS', 'Method 6', 'Target PFA'], value='RSS', description='Method')
gbpfa = widgets.FloatText(value=2, description='Target PFA%')
gbapply = widgets.Button(description='Apply')
gbbox = widgets.VBox([gbmethod, gbpfa, gbapply])
gbaccordion = widgets.Accordion(children=[gbbox], selected_index=None)
gbaccordion.set_title(0, 'Guardbanding')
gbpfa.layout.visibility = 'hidden'
gbmethod.observe(on_gbmethod, names='value')
gbapply.on_click(on_gbapply)

outlayout = widgets.VBox([fig.canvas, tableout])
ctllayout = widgets.VBox([witp, wtur, wgbf, wmeas, gbaccordion])
layout = widgets.HBox([ctllayout, outlayout])
display(layout)

r.out.plot_joint(plot=fig)
with tableout:
    display(r.out)



---

#### Definitions

- **In-tolerance probability**: The probability that any product (manufactured or calibrated) falls within the tolerance limits
- **Test Uncertainty Ratio (TUR)**: Ratio of tolerance to measurement uncertainty at 95% confidence
- **Guardband Factor**: The guardbanded acceptance limit will be (tolerance × guardband factor). Guardband factor of 1 indicates no guardbanding is applied.
- **Measured Value**: The value as measured. Affects Specific Risk, but not Global Risk.
- **Process Risk**: Probability that any product is out of limits, regardless of any inspection/test measurement
- **Specific False Accept Risk**: probability that a product is out of tolerance given the specific measured value result
- **Global False Accept Risk (PFA)**: probability that any product is falsely accepted by an inspection/test measurement
- **Global False Reject Risk (PFR)**: probability that any product is falsely rejected by an inspection/test measurement

#### Guardbanding Methods:

- **RSS**: Calculates guardband factor as $\sqrt{1-1/TUR^2}$
- **Method 6**: Calculates guardband factor as $1 - \frac{1.04 - \exp(0.38 \log(TUR) - 0.54)}{TUR}$ (See Dobbert, "A guardband strategy for managing false-accept risk")
- **Target PFA**: Numerically solve for the guardband factor that results in the desired PFA

---

#### Suncal

Suncal was developed by the Primary Standards Lab at Sandia National Laboratories. For more features and desktop version, see [https://sandiapsl.github.io](https://sandiapsl.github.io).

© 2019-2021 National Technology & Engineering Solutions of Sandia, LLC (NTESS).
Under the terms of Contract DE-NA0003525 with NTESS, the U.S. Government
retains certain rights in this software.